In [2]:
import sys,os
import matk
import numpy as np
from matplotlib import pyplot as plt

# define a simple decaying sinusoidal function
def sine_decay(params, x, data):
    """ model decaying sine wave, subtract data"""
    amp = params['amp']
    shift = params['shift']
    omega = params['omega']
    decay = params['decay']

    model = amp * np.sin(x * omega + shift) * np.exp(-x*x*decay)

    obsnames = ['obs'+str(i) for i in range(1,len(data)+1)]
    return dict(zip(obsnames,model))


# create noisy data
x = np.linspace(0, 15, 301)
np.random.seed(1000)
data = (5. * np.sin(2 * x - 0.1) * np.exp(-x*x*0.025) +
        np.random.normal(size=len(x), scale=0.2) )

# Create MATK object
p = matk.matk(model=sine_decay, model_args=(x,data,))

# Create parameters
p.add_par('amp', value=10, min=0., max=20.)
p.add_par('decay', value=0.1, min=0, max=0.2)
p.add_par('shift', value=0.0, min=-np.pi/2., max=np.pi/2.)
p.add_par('omega', value=3.0, min=0, max=6)

# Create observation names and set observation values to noisy data
for i in range(len(data)):
    p.add_obs('obs'+str(i+1), value=data[i])

# Create LHS sample
ss1 = p.lhs(siz=1000)

# Execute the model on the LHS sample parameter combinations.
ss1.run(verbose=False)

# Perform the RBD_FAST analysis
SS = ss1.rbd_fast()

Parameter First
amp       0.416543
decay     0.289382
shift     0.0286196
omega     0.0386413


In [3]:
# Manually print some results
print SS['S1']

[0.41654295222954379, 0.28938223959466058, 0.028619627269347119, 0.03864126571374104]
